In [ ]:
import geopandas as gpd

In [ ]:
# Ref: https://gf0604-procesamientodatosgeograficos.github.io/2022-i/datos/sinac/areas-silvestres-protegidas.geojson
asp = gpd.read_file("asp.geojson")
# Ensure geometry is in EPSG:4326 for degrees
asp = asp.to_crs("EPSG:4326")
asp

In [ ]:
asp

In [ ]:
# Ensure geometry is in EPSG:4326 for degrees
asp = asp.to_crs("EPSG:4326")

# Calculate centroids for all geometries
asp["centroid"] = asp.geometry.centroid

# # Extract latitude and longitude from centroids
asp["longitude"] = asp.centroid.x
asp["latitude"] = asp.centroid.y


In [ ]:
# Filter by PN
asp_filtered_df = asp[asp['siglas_cat'] == 'PN']
asp_filtered_df = asp_filtered_df[~asp_filtered_df['descripcio'].str.contains('marina', na=False)]

asp_filtered_df

In [ ]:
asp_filtered_point =asp_filtered_df.drop(columns="centroid")

In [ ]:
import geemap
for i in range(len(asp_filtered_point)):  # Loop through row indices
    chunk = asp_filtered_point.iloc[[i]]  # Keep it as a GeoDataFrame

    # Convert to GeoJSON format
    geojson_data = geemap.gdf_to_geojson(chunk)

    # Save as a GeoJSON file
    output_file = f"output_chunk_{i}.geojson"
    chunk.to_file(output_file, driver="GeoJSON")

    print(f"Saved: {output_file}")

In [ ]:
import geemap

Map = geemap.Map()
Map.add_gdf(asp_filtered_point)
Map.add_geojson(geojson_data, "SUUUUP")
Map

In [ ]:
asp.to_csv("asp_lat_long.csv", index=False)


In [ ]:
# Get PN Only
import geemap
from geojson import Point, Feature, FeatureCollection, dump
from shapely.wkt import dumps



asp_pn = asp[asp['siglas_cat'] == "PN"]

asp_pn = asp_pn.sort_values(by="codigo", ascending=True)
asp_pn["geometry"] = asp_pn["geometry"].apply(dumps)  # Convert geometry to WKT
asp_pn.to_csv("output_with_geometry.csv", index=False)


In [ ]:
import geemap
from geojson import Point, Feature, FeatureCollection, dump

asp_pn = asp_pn.to_crs("EPSG:4326")
asp_pn.geometry = asp_pn.geometry.make_valid()
fil = asp_pn.iloc[0:1]
fil['geometry'] = fil['geometry'].simplify(0.1).buffer(0)
geojson = geemap.gdf_to_geojson(fil)
with open('myfile.geojson', 'w') as f:
   dump(geojson, f)

In [ ]:
import geemap
a = geemap.gdf_to_geojson(asp_pn)
a

In [ ]:
# Set the chunk size
chunk_size = 5

# Split the GDF into chunks and save each as a separate CSV
for i in range(0, len(asp_pn), chunk_size):
    chunk = asp_pn.iloc[i:i + chunk_size]
    chunk.to_csv(f"output_chunk_{i // chunk_size + 1}.csv", index=False)


In [ ]:
asp_filtered_df

In [ ]:
import json
from shapely.geometry import mapping

# Assume your GeoDataFrame is named gdf and has a column 'name'
# Create a new column 'raw_geojson' that holds the GeoJSON string for each geometry.
asp_filtered_df['raw_geojson'] = asp_filtered_df.geometry.apply(lambda geom: json.dumps(mapping(geom)))

# Select only the necessary columns
output_df = asp_filtered_df[['nombre_asp', 'raw_geojson']]

# Export to CSV
output_df.to_csv('output.csv', index=False)
